In [1]:
import ocsmesh
import geopandas as gpd
from pyproj import CRS,Transformer
from shapely import intersection
import pandas as pd
from stormevents.nhc import VortexTrack
from copy import deepcopy
from ocsmesh import Hfun, Mesh, JigsawDriver,Geom
import numpy as np
from scipy.interpolate import griddata
from shapely.ops import polygonize, unary_union, transform
from shapely.geometry import ( # type: ignore[import]
        Polygon, MultiPolygon,
        box, GeometryCollection, Point, MultiPoint,
        LineString, LinearRing)
import utm

In [2]:
def _calculate_mesh_size_function(
        buffer_domain,
        hires_mesh_clip,
        lores_mesh_clip,
        buffer_crs
    ):

    assert buffer_crs == hires_mesh_clip.crs == lores_mesh_clip.crs

    # HARDCODED FOR NOW
    approx_elem_per_width = 3

    msht_hi = deepcopy(hires_mesh_clip)
    msht_lo = deepcopy(lores_mesh_clip)

    crs = buffer_crs
    assert(not buffer_crs.is_geographic)

    # calculate mesh size for clipped bits
    hfun_hi = Hfun(Mesh(msht_hi))
    hfun_hi.size_from_mesh()

    hfun_lo = Hfun(Mesh(msht_lo))
    hfun_lo.size_from_mesh()

    msht_cdt = ocsmesh.utils.triangulate_polygon(
        buffer_domain, None, opts='p'
    )
    msht_cdt.crs = crs

    hfun_cdt = Hfun(Mesh(msht_cdt))
    hfun_cdt.size_from_mesh()

    hfun_cdt_sz = deepcopy(hfun_cdt.msh_t().value) / approx_elem_per_width
    msht_cdt.value[:] = hfun_cdt_sz
    hfun_rep = Hfun(Mesh(msht_cdt))

    mesh_domain_rep = JigsawDriver(
        geom=Geom(buffer_domain, crs=crs),
        hfun=hfun_rep,
        initial_mesh=False
    ).run(sieve=0)

    msht_domain_rep = deepcopy(mesh_domain_rep.msh_t)
#        utils.reproject(msht_domain_rep, crs)

    pts_2mesh = np.vstack(
        (hfun_hi.msh_t().vert2['coord'], hfun_lo.msh_t().vert2['coord'])
    )
    val_2mesh = np.vstack(
        (hfun_hi.msh_t().value, hfun_lo.msh_t().value)
    )
    domain_sz_1 = griddata(
        pts_2mesh, val_2mesh, msht_domain_rep.vert2['coord'], method='linear'
    )
    domain_sz_2 = griddata(
        pts_2mesh, val_2mesh, msht_domain_rep.vert2['coord'], method='nearest'
    )
    domain_sz = domain_sz_1.copy()
    domain_sz[np.isnan(domain_sz_1)] = domain_sz_2[np.isnan(domain_sz_1)]

    msht_domain_rep.value[:] = domain_sz
    hfun_interp = Mesh(msht_domain_rep)

    return hfun_interp

In [3]:
def get_shape(storm, year):

    track = VortexTrack.from_storm_name(
        storm, year, file_deck='b', advisories=['BEST']
    )

    ws34 = track.wind_swaths(wind_speed=34)
    # Get single best track
    best_ws34 = list(ws34['BEST'].values())[0]

    return best_ws34

In [4]:
shape = get_shape('beryl', 2024)

highres = ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/STOFS-3D-Atlantic_v2.1.gr3', crs=4326)
lowres = ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/WNAT_1km.14', crs=4326)

highres_clip = ocsmesh.utils.clip_mesh_by_shape(
    highres.msh_t,
    shape=shape,
    inverse=False,
    fit_inside=False,
    check_cross_edges=False,
    adjacent_layers=0
)


C:\Users\Felicio.Cassalho\.conda\envs\model_vis\lib\site-packages\stormevents\nhc\track.py:1109: RuntimeWarning: invalid value encountered in divide
  speeds = pandas.Series(distances / abs(intervals), index=indices)


In [5]:
ocsmesh.Mesh(lowres.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/lowres.2dm", format='2dm', overwrite=True)
ocsmesh.Mesh(highres.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/highres.2dm", format='2dm', overwrite=True)

In [6]:
ocsmesh.Mesh(highres_clip).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/highres_clip.2dm", format='2dm', overwrite=True)

In [7]:
shape_gdf = gpd.GeoDataFrame(
    geometry = [shape],crs=4326)
# shape_gdf.to_file(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/shape.shp")

In [8]:
merged = ocsmesh.utils.merge_overlapping_meshes(
    [lowres.msh_t, highres_clip], adjacent_layers=10
)

ocsmesh.utils.cleanup_duplicates(merged)
ocsmesh.utils.put_id_tags(merged)

In [9]:
ocsmesh.Mesh(merged).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/merged.2dm", format='2dm', overwrite=True)

In [11]:
### create_mesh_from_mesh_diff

In [200]:
carved_mesh = ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\Pearl_River\temp/carved_mesh.2dm', crs=4326).msh_t

In [201]:
domain_start=[highres_clip,lowres.msh_t]
mesh_1 = carved_mesh
mesh_2 = highres_clip
hfun_mesh = None
crs=CRS.from_epsg(4326)
min_int_ang=30
buffer_domain = 0.01

In [10]:
domain = gpd.GeoDataFrame(geometry=[ocsmesh.utils.get_mesh_polygons(i) for i in domain_start],crs=crs)

In [12]:
domain_buffer = gpd.GeoDataFrame(geometry=[i[-1].geometry.buffer(buffer_domain) for i in domain.iterrows()],crs=crs)
domain_buffer = domain_buffer.dissolve().explode(index_parts=True)
domain_buffer.crs = domain_buffer.estimate_utm_crs()
domain_buffer =domain_buffer.loc[domain_buffer['geometry'].area == max(domain_buffer['geometry'].area)]
domain_buffer.crs = crs
domain_buffer = gpd.GeoDataFrame(geometry=[domain_buffer.union_all()],crs=crs)

In [13]:
poly_buffer = domain_buffer.union_all().difference(
    gpd.GeoDataFrame(
        geometry=[
            ocsmesh.utils.get_mesh_polygons(mesh_1),
            ocsmesh.utils.get_mesh_polygons(mesh_2),
        ],
        crs = crs
    ).union_all()
)
gdf_full_buffer = gpd.GeoDataFrame(
    geometry = [poly_buffer],crs=crs)

In [14]:
clipper = domain.union_all().difference(ocsmesh.utils.get_mesh_polygons(mesh_1)).difference(ocsmesh.utils.get_mesh_polygons(mesh_2))

### This is the new function, for creating hfun, this is done outside create_mesh_from_mesh_diff

In [ ]:
mesh_1_dc = deepcopy(mesh_1)
mesh_2_dc = deepcopymesh_2)

In [36]:
utm = ocsmesh.utils.estimate_bounds_utm(clipper.bounds, 4326)
# Transform all inputs to UTM:
t1 = Transformer.from_crs(4326, utm, always_xy=True)
clipper_utm = transform(t1.transform, clipper)
ocsmesh.utils.reproject(mesh_2_dc, utm)
ocsmesh.utils.reproject(mesh_1_dc, utm)

In [38]:
hfun_buffer = _calculate_mesh_size_function(
        clipper_utm,
        mesh_2_dc,
        mesh_1_dc,
        mesh_1_dc.crs
    )

In [39]:
ocsmesh.Mesh(hfun_buffer.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_buffer_notutm.2dm", format='2dm', overwrite=True)

In [64]:
hfun=Hfun(hfun_buffer)
mesh_buf_apprx = JigsawDriver(
            geom=Geom(ocsmesh.utils.get_mesh_polygons(hfun_buffer.msh_t), crs=mesh_2_dc.crs),
            hfun=hfun,
            initial_mesh=False
        ).run(sieve=0)

ocsmesh.utils.reproject(mesh_buf_apprx.msh_t, lowres.msh_t.crs)

In [68]:
ocsmesh.Mesh(mesh_buf_apprx.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/mesh_buf_apprx_domain_final.2dm", format='2dm', overwrite=True)

#### end of hfun func

##### If the uses passes a mesh to be the hfun, instead of creating, that should be ok. Inside the create_mesh_from_mesh_diff we should clip the hfun (given or created) with the buffer.
##### So, if larger, then no problem/

In [143]:
hfun_mesh = mesh_buf_apprx.msh_t

In [163]:
# hfun_clip = ocsmesh.utils.clip_mesh_by_shape(hfun_mesh, clipper, check_cross_edges=True)
hfun_clip = hfun_mesh

In [164]:
ocsmesh.Mesh(hfun_clip).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_clip.2dm", format='2dm', overwrite=True)

In [165]:
boundary = np.unique(ocsmesh.utils.get_boundary_edges(hfun_clip))
all_nodes = hfun_clip.vert2['coord']

In [166]:
# arr = np.array(all_nodes)
arr = np.delete(all_nodes, boundary, axis=0)

In [170]:
points = gpd.GeoDataFrame(geometry = [MultiPoint(arr)],crs=crs)

In [171]:
root_dir = r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/"

In [172]:
points.to_file(root_dir+'points.shp')

In [175]:
msht_buffer = ocsmesh.utils.triangulate_polygon(gdf_full_buffer,
                                    # min_int_ang=30,
                                    aux_pts=arr)

In [176]:
#ocsmesh.Mesh(msht_buffer).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/msht_buffer.2dm", format='2dm', overwrite=True)

In [177]:
msht_buffer = ocsmesh.utils.clip_mesh_by_shape(msht_buffer,clipper)

In [192]:
msht_buffer.crs = crs

In [193]:
ocsmesh.Mesh(msht_buffer).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/msht_buffer.2dm", format='2dm', overwrite=True)

In [206]:
merged = ocsmesh.utils.merge_neighboring_meshes(msht_buffer,mesh_1)

In [207]:
ocsmesh.Mesh(merged).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/merged.2dm", format='2dm', overwrite=True)

In [ ]:
boundary = np.unique(ocsmesh.utils.get_boundary_edges(mesh_buf_apprx))
all_nodes = mesh_buf_apprx.vert2['coord']
arr = np.delete(all_nodes, boundary, axis=0)

In [ ]:
clipper = clipper.union_all().difference(ocsmesh.utils.get_mesh_polygons(mesh_1)).difference(ocsmesh.utils.get_mesh_polygons(mesh_2))

In [ ]:
domain_cleanup = gpd.GeoDataFrame(geometry=
                    gpd.GeoSeries(intersection(
                        ocsmesh.utils.get_mesh_polygons(domain_start[0]),
                        ocsmesh.utils.get_mesh_polygons(domain_start[1]),
                        ))).dissolve().buffer(buffer_domain*2)
domain_cleanup = ocsmesh.utils.remove_holes(domain_cleanup.union_all())
domain_cleanup = gpd.GeoDataFrame(geometry=
                                 gpd.GeoSeries(domain_cleanup),
                                 crs=4326).dissolve()
domain = pd.concat([gpd.GeoDataFrame\
                    (geometry=[ocsmesh.utils.get_mesh_polygons(i)\
                               .buffer(buffer_domain,join_style=2)
                               ],\
                                crs=crs) for i in domain_start])


In [ ]:
domain = domain.dissolve().explode(index_parts=True)
domain.crs = domain.estimate_utm_crs()
domain =domain.loc[domain['geometry'].area == max(domain['geometry'].area)]
domain.crs = crs
domain = gpd.GeoDataFrame(geometry=[domain.union_all()],crs=crs)

In [ ]:
poly_buffer = domain.union_all().difference(
    gpd.GeoDataFrame(
        geometry=[
            ocsmesh.utils.get_mesh_polygons(mesh_1),
            ocsmesh.utils.get_mesh_polygons(mesh_2),
        ],
        crs = crs
    ).union_all()
)
gdf_full_buffer = gpd.GeoDataFrame(
    geometry = [poly_buffer],crs=crs)

In [6]:
root_dir = r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/"

In [22]:
gdf_full_buffer.to_file(root_dir+"temp/gdf_full_buffer.shp") 
domain.to_file(root_dir+"temp/domain.shp") 
domain_cleanup.to_file(root_dir+"temp/domain_cleanup.shp") 

In [86]:
carved_mesh = ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\Pearl_River\temp/carved_mesh.2dm', crs=4326).msh_t
highres_clip =ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/highres_clip.2dm', crs=4326).msh_t
lowhres =ocsmesh.Mesh.open(r'C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/lowres.2dm', crs=4326).msh_t

In [11]:
domain_final = gpd.GeoDataFrame(
        geometry=[
            ocsmesh.utils.get_mesh_polygons(highres_clip),
            ocsmesh.utils.get_mesh_polygons(lowhres),
        ],
        crs = 4326
    )
domain_final = domain_final.union_all().difference(ocsmesh.utils.get_mesh_polygons(carved_mesh)).difference(ocsmesh.utils.get_mesh_polygons(highres_clip))

In [15]:
gpd.GeoDataFrame(geometry = [domain_final],crs=4326).to_file(root_dir+"temp/domain_final.shp") 

In [17]:
poly_isotach = domain_final

In [18]:
utm = ocsmesh.utils.estimate_bounds_utm(poly_isotach.bounds, 4326)

In [20]:
utm = ocsmesh.utils.estimate_bounds_utm(poly_isotach.bounds, 4326)
# Transform all inputs to UTM:
t1 = Transformer.from_crs(4326, utm, always_xy=True)
poly_isotach = transform(t1.transform, poly_isotach)
ocsmesh.utils.reproject(highres_clip, utm)
ocsmesh.utils.reproject(carved_mesh, utm)

In [21]:
hfun_buffer = _calculate_mesh_size_function(
        poly_isotach,
        highres_clip,
        carved_mesh,
        carved_mesh.crs
    )

In [22]:
hfun=Hfun(hfun_buffer)

In [23]:
ocsmesh.Mesh(hfun_buffer.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_buffer_domain_final.2dm", format='2dm', overwrite=True)

In [31]:
hfun_buffer = ocsmesh.Mesh.open(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_buffer_domain_final.2dm", crs=4326).msh_t

In [34]:
hfun=Hfun(Mesh(hfun_buffer))

In [35]:
hfun=Hfun(Mesh(hfun_buffer))
mesh_buf_apprx = JigsawDriver(
            geom=Geom(ocsmesh.utils.get_mesh_polygons(hfun_buffer), crs=4326),
            hfun=hfun,
            initial_mesh=False
        ).run(sieve=0)

In [36]:
ocsmesh.Mesh(mesh_buf_apprx.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/mesh_buf_apprx_domain_final.2dm", format='2dm', overwrite=True)

In [47]:
mesh_buf_apprx = ocsmesh.Mesh.open(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/mesh_buf_apprx_domain_final.2dm", crs=4326).msh_t

In [48]:
boundary = np.unique(ocsmesh.utils.get_boundary_edges(mesh_buf_apprx))

In [49]:
all_nodes = mesh_buf_apprx.vert2['coord']

In [50]:
arr = np.delete(all_nodes, boundary, axis=0)

In [53]:
final_buffer_mesh = ocsmesh.utils.triangulate_polygon(shape=gdf_full_buffer,aux_pts=arr,opts='p')

In [54]:
ocsmesh.Mesh(final_buffer_mesh).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/final_buffer_mesh.2dm", format='2dm', overwrite=True)

C:\Users\Felicio.Cassalho\.conda\envs\model_vis\lib\site-packages\ocsmesh\mesh\mesh.py:106: UserWarning: Input mesh has no CRS information.
  warnings.warn('Input mesh has no CRS information.')


In [76]:
final_buffer_mesh_clipped = ocsmesh.utils.clip_mesh_by_shape(final_buffer_mesh,domain_final,adjacent_layers=1)

In [77]:
final_buffer_mesh_clipped.crs = mesh_buf_apprx.crs

In [78]:
ocsmesh.Mesh(final_buffer_mesh_clipped).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/final_buffer_mesh_clipped.2dm", format='2dm', overwrite=True)

In [90]:
final_merged = ocsmesh.utils.merge_neighboring_meshes(highres_clip,final_buffer_mesh_clipped,carved_mesh)

In [91]:
ocsmesh.Mesh(final_merged).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/final_merged.2dm", format='2dm', overwrite=True)

In [8]:
gdf_full_buffer = gpd.read_file(root_dir+"temp/gdf_full_buffer.shp") 
domain_cleanup = gpd.read_file(root_dir+"temp/domain_cleanup.shp") 

In [9]:
poly_isotach = intersection(gdf_full_buffer.union_all(), domain_cleanup.union_all())

In [10]:
utm = ocsmesh.utils.estimate_bounds_utm(poly_isotach.bounds, 4326)

In [11]:
# Transform all inputs to UTM:
t1 = Transformer.from_crs(4326, utm, always_xy=True)
poly_isotach = transform(t1.transform, poly_isotach)
ocsmesh.utils.reproject(highres_clip, utm)
ocsmesh.utils.reproject(carved_mesh, utm)

In [ ]:
hfun_buffer = _calculate_mesh_size_function(
        poly_isotach,
        highres_clip,
        carved_mesh,
        carved_mesh.crs
    )

In [ ]:
hfun=Hfun(hfun_buffer)

In [ ]:
hfun_buffer.msh_t

In [56]:
ocsmesh.Mesh(hfun_buffer.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_buffer.2dm", format='2dm', overwrite=True)

In [5]:
hfun_buffer = ocsmesh.Mesh.open(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/hfun_buffer.2dm", crs=4326).msh_t

In [6]:
root_dir = r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/"
gdf_full_buffer = gpd.read_file(root_dir+"temp/gdf_full_buffer.shp") 
domain_cleanup = gpd.read_file(root_dir+"temp/domain_cleanup.shp") 

In [59]:
hfun_buffer = ocsmesh.utils.clip_mesh_by_shape(hfun_buffer,domain_cleanup.union_all())

In [60]:
hfun=Hfun(Mesh(hfun_buffer))

In [61]:
mesh_buf_apprx = JigsawDriver(
            geom=Geom(ocsmesh.utils.get_mesh_polygons(hfun_buffer), crs=4326),
            hfun=hfun,
            initial_mesh=False
        ).run(sieve=0)

In [62]:
ocsmesh.Mesh(mesh_buf_apprx.msh_t).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/mesh_buf_apprx.2dm", format='2dm', overwrite=True)

In [94]:
boundary = ocsmesh.utils.get_boundary_edges(mesh_buf_apprx.msh_t).ravel()
boundary_removed = ocsmesh.utils.remove_mesh_by_edge(mesh_buf_apprx.msh_t,boundary)

In [95]:
boundary = ocsmesh.utils.get_boundary_edges(boundary_removed).ravel()
boundary_removed = ocsmesh.utils.remove_mesh_by_edge(boundary_removed,boundary)

In [96]:
ocsmesh.Mesh(boundary_removed).write(r"C:\Users\Felicio.Cassalho\Work\Python_Development\Mesh\end-to-end\subsetting/temp/boundary_removed.2dm", format='2dm', overwrite=True)

In [ ]:
final_domain = 